In [1]:
import numpy as np
import pandas as pd

In [ ]:
#first, correct cancer files read count biases

In [2]:
dilution_calc = pd.read_excel('C:\\Users\\mayh\\OneDrive - A STAR\\atac_seq\\model\\dilution_calc.xlsx', sheet_name='train')
dilution_calc

,healthy_file_name,healthy_read_counts,cancer_file_label,cancer_file_name,original,cancer_read_counts,correction_factor,corrected read counts
0,WHC556,2061558900,CRC-1,1014_180816,0.455,2224638397,0.926694,2.061559e+09
1,WHC556,2061558900,CRC-2,1531_160616,0.858,2178637561,0.946261,2.061559e+09
2,WHC556,2061558900,CRC-3,1531_180119,0.490,2364417111,0.871910,2.061559e+09
3,WHC556,2061558900,CRC-4,519_210114,0.351,2292445291,0.899284,2.061559e+09
4,WHC556,2061558900,CRC-5,809_030915,0.590,2227096655,0.925671,2.061559e+09
...,...,...,...,...,...,...,...,...
91,healthy total,19726151323,CRC-4,519_210114,0.351,2292445291,8.604852,1.972615e+10
92,healthy total,19726151323,CRC-5,809_030915,0.590,2227096655,8.857340,1.972615e+10
93,healthy total,19726151323,CRC-6,809_110914,0.469,2305429143,8.556390,1.972615e+10
94,healthy total,19726151323,CRC-7,986_100215,0.419,2229529895,8.847673,1.972615e+10


In [3]:
num_list_h = ['556', '557', '558', '559', '560', '561', '562', '563', '564', '565', '566']
num_list_c = ['1014_180816', '1531_160616', '1531_180119', '519_210114', '809_030915', '809_110914', '986_100215', '986_261016']

In [4]:
with open('C:\\Users\\mayh\\OneDrive - A STAR\\atac_seq\\model\\\score\\healthy_list_altered.txt') as file:
    regions_h = [line.rstrip() for line in file]
len(regions_h)

144

In [5]:
regions_c = [region[3:] for region in regions_h]
len(regions_c)

144

In [6]:
for num_c in num_list_c:
    for num_h in num_list_h:
        for region in regions_c:
            fn = 'C:\\Users\\mayh\\OneDrive - A STAR\\atac_seq\\model\\cancer_raw_train\\selected-raw-cancer-' + \
                 num_c + '-' + region
            df = pd.read_csv(fn, header=None, sep='\t')
            factor = dilution_calc[(dilution_calc.healthy_file_name == 'WHC' + num_h) & \
                              (dilution_calc.cancer_file_name == num_c)]['correction_factor'].values[0]
            df[2] = df[2] * factor
            df.to_csv('C:\\Users\\mayh\\OneDrive - A STAR\\atac_seq\\model\\cancer_raw_train\\' + num_c + \
                      '\\WHC' + num_h + '\\' + region)

In [7]:
#second, mix!

In [8]:
file_origin_column = []
fraction_column = []
regions_dic = {}
for region_c in regions_c:
    region_h = 'chr' + region_c
    region_column = []
    for num_c in num_list_c:
        for num_h in num_list_h:
            frac_calc = pd.read_excel('C:\\Users\\mayh\\OneDrive - A STAR\\atac_seq\\model\\dilution_calc.xlsx', sheet_name = num_c)
            fn_c = 'C:\\Users\\mayh\\OneDrive - A STAR\\atac_seq\\model\\cancer_raw_train\\' + num_c + \
                    '\\WHC' + num_h + '\\' + region_c
            fn_h = 'C:\\Users\\mayh\\OneDrive - A STAR\\atac_seq\\model\\healthy_raw_train\\selected-raw-healthy-WHC' + \
                    num_h + '-' + region_h
            raw_c = pd.read_csv(fn_c)['2']
            raw_h = pd.read_csv(fn_h, header = None, sep = '\t')[2]
            for fraction in frac_calc['target_fraction']:
                if len(regions_dic.keys()) == 0:
                    file_origin_column.append(num_c + num_h)
                    fraction_column.append(fraction)
                
                healthy_factor = frac_calc[frac_calc.target_fraction == fraction]['healthy_factor'].values[0]
                cancer_factor = frac_calc[frac_calc.target_fraction == fraction]['cancer_factor'].values[0]
                
                raw_mixed = cancer_factor * raw_c + healthy_factor * raw_h
                left_flank_sum = raw_mixed[0:5000].sum()
                right_flank_sum = raw_mixed[7000:12000].sum()
                flank_mean = (left_flank_sum + right_flank_sum) / 10000
                avg = (raw_mixed[5900:6100] / flank_mean).mean()
                
                region_column.append(avg)
    regions_dic[region_c] = region_column

In [9]:
regions_df = pd.DataFrame(regions_dic)
regions_df['fraction'] = fraction_column
regions_df['file_origin'] = file_origin_column
regions_df

,2_90377309-90389308,10_115913313-115925312,5_132088994-132100993,12_76707915-76719914,9_111201851-111213850,11_94314325-94326324,1_86102052-86114051,4_49653612-49665611,11_4191008-4203007,10_34710330-34722329,...,X_10898615-10910614,5_7165158-7177157,4_4085940-4097939,4_120310198-120322197,7_152659206-152671205,6_55727191-55739190,6_47483844-47495843,2_163679216-163691215,fraction,file_origin
0,3.622756,1.264198,1.162033,1.347972,0.782820,1.138017,1.231842,20.903026,1.139571,1.176411,...,1.082682,1.503277,0.814309,1.147954,1.163289,1.245935,1.492135,1.448191,0.000,1014_180816556
1,3.616985,1.257997,1.159546,1.342193,0.782499,1.133273,1.229751,20.859483,1.135162,1.177106,...,1.081346,1.500096,0.813355,1.145272,1.162195,1.245070,1.487463,1.439935,0.005,1014_180816556
2,3.611258,1.251769,1.157047,1.336395,0.782177,1.128521,1.227644,20.815990,1.130758,1.177804,...,1.079994,1.496906,0.812393,1.142579,1.161095,1.244204,1.482778,1.431693,0.010,1014_180816556
3,3.605576,1.245513,1.154537,1.330579,0.781853,1.123760,1.225521,20.772545,1.126359,1.178505,...,1.078627,1.493707,0.811424,1.139876,1.159990,1.243338,1.478078,1.423466,0.015,1014_180816556
4,3.599938,1.239230,1.152014,1.324743,0.781528,1.118991,1.223381,20.729150,1.121964,1.179209,...,1.077244,1.490497,0.810447,1.137163,1.158878,1.242470,1.473364,1.415253,0.020,1014_180816556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2624,3.452866,0.818405,0.938359,1.042517,0.901093,1.091786,0.950860,20.544299,0.957624,0.942435,...,0.909932,0.954428,0.770162,0.950669,1.099212,1.207809,0.993408,1.023863,0.350,986_261016566
2625,3.418379,0.801174,0.936425,1.001330,0.887621,1.086704,0.917080,20.132160,0.935014,0.938504,...,0.895594,0.914253,0.762175,0.957061,1.073728,1.208863,0.984272,1.032757,0.375,986_261016566
2626,3.381558,0.783298,0.934360,0.959049,0.873725,1.081555,0.882191,19.723818,0.912006,0.934434,...,0.880844,0.873000,0.753797,0.963760,1.047521,1.209923,0.974952,1.041924,0.400,986_261016566
2627,3.342158,0.764738,0.932151,0.915630,0.859385,1.076339,0.846139,19.319219,0.888590,0.930215,...,0.865663,0.830627,0.744998,0.970786,1.020559,1.210991,0.965441,1.051377,0.425,986_261016566


In [10]:
regions_df.to_csv('train_new.csv')